In [ ]:
import wandb
import torch
import pytorch_lightning as pl
from dataset import make_dataloaders, make_dataloaders_celeba
from PLModel import PLColorDiff
from model import ColorDiffusion
from unet import SimpleUnet
from utils import get_device
from pytorch_lightning.loggers import WandbLogger
from icecream import ic
import default_configs
# from stable_diffusion.model.unet import UNetModel
from denoising import Unet, Encoder
from pytorch_lightning.callbacks import ModelCheckpoint

# colordiff_config = dict(
#     device = "gpu",
#     pin_memory = True,
#     T=350,
#     # lr=6e-4,
#     lr = 5e-6,
#     loss_fn = "l2",
#     batch_size=18,
#     accumulate_grad_batches=4,
#     img_size = 128,
#     sample=True,
#     should_log=True,
#     epochs=200,
#     using_cond=True,
#     display_every=100,
#     dynamic_threshold=False,
#     train_autoenc=False,
#     enc_loss_coeff = 1.1,
# ) 
colordiff_config = dict(
    device = "gpu",
    pin_memory = True,
    T=350,
    # lr=6e-4,
    lr = 7e-5,
    loss_fn = "l2",
    batch_size=72,
    accumulate_grad_batches=1,
    img_size = 64,
    sample=True,
    should_log=True,
    epochs=200,
    using_cond=True,
    display_every=100,
    dynamic_threshold=False,
    train_autoenc=False,
    enc_loss_coeff = 1.1,
) 

if __name__ == "__main__":
    colordiff_config["device"] = "gpu" 
    train_dl, val_dl = make_dataloaders_celeba("./img_align_celeba", colordiff_config, num_workers=2, limit=20000)
    log = True

    unet_config = dict(
        channels=3,
        dropout=0.3,
        self_condition=False,
        out_dim=2,
        dim=128,
        condition=True,
        dim_mults=[1, 2, 3, 3],
    )
    enc_config = dict(
        channels=1,
        dropout=0.3,
        self_condition=False,
        out_dim=2,
        dim=128,
        dim_mults=[1, 2, 3, 3],
    )


    encoder = Encoder(
        **enc_config
    )
    unet = Unet(
        **unet_config,
    ) 
    model = ColorDiffusion.load_from_checkpoint("run_64_good/checkpoints/last.ckpt", strict=False, unet=unet, encoder=encoder, train_dl=train_dl, val_dl=val_dl, **colordiff_config)